# Natural Language Processing - lab 5 (Morphosyntactic tagging)

Bartosz Klimza

# Necessary imports

In [49]:
import os
import requests
import collections as C
import ast
import math


# 3. Use the tool to tag and lemmatize the law corpus:

I used docker image of KRNNT2.

In [2]:
data_dir = "./ustawy/"

words = dict()
for file in os.listdir(data_dir):
    bills = open(f"./{data_dir}{file}", "r").read()
    bills = bills.split()
    bills = " ".join(bills)
    bills = bills.lower()
    req = requests.post("http://localhost:9200", data=bills)
    lines, tags, c = req.bills.split("\n"), [], 0
    while c < len(lines):
        if c + 1 == len(lines): c += 1
            continue
        tag = lines[c + 1]
        tag.split()
        tag_tuple = (tag[0], tag[1])
        tags.append(tag_tuple)
        c += 2
    words[file] = tags


This operation had taken a lof of time so I decided to save the result in txt file

In [3]:
with open("corpus.txt", "w") as f:
    f.write(tag_dict, f)


# 4. Using the tagged corpus compute bigram statistic for the tokens containing:
* lemmatized, downcased word,
* morphosyntactic category of the word (subst, fin, adj, etc.)

In [42]:
results = C.Counter(dict())

def corpus_bigrams(words):
    bigrams = []
    for w in words:
        token, rest = w[0], w[1].split(":")[0]
        bigrams.append((token, rest))
        result_list = []
        for i in range(len(words) - 1):
            result_list.append(tuple(bigrams[i:i + 2]))
    return result_list

with open("corpus.txt", 'r') as f:
    file = f.read()
    file_dict = ast.literal_eval(file)
    for v in file_dict.values():
        results += C.Counter(corpus_bigrams(v))


# 5. Discard bigrams containing characters other than letters. Make sure that you discard the invalid entries after computing the bigram counts:

In [48]:
for k in results.keys():
    w1, w2 = k.split(":")[0], k.split(" ")[1].split(":")[0]
    if not w1.isalpha():
        del results[k]
    if not w2.isalpha():
        del results[k]


In [47]:
results_sorted = {k: v for k, v in sorted(results.items(), key=lambda item: item[1], reverse=True)}
results_sorted


{'w:prep artykuł:brev': 31972,
 'o:prep który:adj': 28656,
 'który:adj mowa:subst': 28538,
 'mowa:subst w:prep': 28473,
 'w:prep ustęp:brev': 23501,
 'z:prep dzień:subst': 11360,
 'otrzymywać:fin brzmienie:subst': 10532,
 'określić:ppas w:prep': 9772,
 'do:prep sprawa:subst': 8718,
 'ustawa:subst z:prep': 8625,
 'właściwy:adj do:prep': 8535,
 'i:conj numer:brev': 8435,
 'dodawać:fin się:qub': 8196,
 'minister:subst właściwy:adj': 7933,
 'w:prep brzmienie:subst': 7280,
 'w:prep droga:subst': 7128,
 'w:prep przypadek:subst': 6776,
 'na:prep podstawa:subst': 6681,
 'stosować:fin się:qub': 6535,
 'się:qub wyraz:subst': 6077,
 'w:prep zakres:subst': 5476,
 'od:prep dzień:subst': 5324,
 'w:prep ustawa:subst': 5276,
 'zastępować:fin się:qub': 4779,
 'droga:subst rozporządzenie:subst': 4747,
 'móc:fin być:inf': 4551,
 'w:prep sprawa:subst': 4500,
 'oraz:conj z:prep': 4302,
 'w:prep życie:subst': 4289,
 'w:prep termin:subst': 4190,
 'a:conj także:conj': 3779,
 'w:prep punkt:brev': 3761,
 'rzecz

# 7. Compute LLR statistic for this dataset:

In [51]:
def llr_2x2(k11, k12, k21, k22):
    '''Special case of llr with a 2x2 table'''
    return 2 * (denormEntropy([k11 + k12, k21 + k22]) +
                denormEntropy([k11 + k21, k12 + k22]) -
                denormEntropy([k11, k12, k21, k22]))

def denormEntropy(counts):
    '''Computes the entropy of a list of counts scaled by the sum of the counts. If the inputs sum to one, this is just the normal definition of entropy'''
    counts = list(counts)
    total = float(sum(counts))
    # Note tricky way to avoid 0*log(0)
    return -sum([k * math.log(k / total + (k == 0)) for k in counts])

w1_dict, w2_dict, llr_dict = dict(), dict(), dict()
bigrams_sum = sum(results_sorted.values())

for k in results_sorted.keys():
    w1, w2 = k.split(":")[0], k.split(" ")[1].split(":")[0]
    if w1 not in w1_dict.keys():
        bigrams = set()
    else:
        bigrams = w1_dict[w1]
    bigrams.add(k)
    w1_dict[w1] = bigrams
    if w2 not in w2_dict.keys():
        bigrams = set()
    else:
        bigrams = w2_dict[w2]
    bigrams.add(k)
    w2_dict[w2] = bigrams

for k, v in results_sorted.items():
    w1, w2 = k.split(":")[0], k.split(" ")[1].split(":")[0]
    b1, b2 = w1_dict[w1], w2_dict[w2]
    k11 = v
    k12 = sum(results_sorted[i] for i in b1) - v
    k21 = sum(results_sorted[i] for i in b2) - v
    k22 = bigrams_sum - k11 - k12 - k21
    llr_dict[k] = llr_2x2(k11, k12, k21, k22)

llr_dict


{'w:prep artykuł:brev': 112549.15804722742,
 'o:prep który:adj': 190482.30155780376,
 'który:adj mowa:subst': 248059.45596041984,
 'mowa:subst w:prep': 177055.41246163473,
 'w:prep ustęp:brev': 87667.14525039634,
 'z:prep dzień:subst': 53432.0298995066,
 'otrzymywać:fin brzmienie:subst': 108829.83895029046,
 'określić:ppas w:prep': 47068.70466282638,
 'do:prep sprawa:subst': 46295.45461158047,
 'ustawa:subst z:prep': 41880.86920423864,
 'właściwy:adj do:prep': 50251.33260918455,
 'i:conj numer:brev': 51516.76198657253,
 'dodawać:fin się:qub': 66404.37807946833,
 'minister:subst właściwy:adj': 70826.57787059317,
 'w:prep brzmienie:subst': 15950.46866787807,
 'w:prep droga:subst': 31970.640096291667,
 'w:prep przypadek:subst': 27589.471751739038,
 'na:prep podstawa:subst': 47037.70995206374,
 'stosować:fin się:qub': 43937.723082786426,
 'się:qub wyraz:subst': 40439.82487653138,
 'w:prep zakres:subst': 18104.945561987348,
 'od:prep dzień:subst': 31543.636325813946,
 'w:prep ustawa:subst':

# 8. Partition the entries based on the syntactic categories of the words, i.e. all bigrams having the form of w1:adj w2:subst should be placed in one partition (the order of the words may not be changed):

In [70]:
partitions = dict()
for k, v in results_sorted.items():
    c1, c2 = k.split(" ")[0].split(":")[1], k.split(" ")[1].split(":")[1]
    morph_cat = (c1, c2)
    if morph_cat in partitions.keys():
        partitions[morph_cat] += v
    else:
        partitions.update({morph_cat: v})

partitions


{('prep', 'brev'): 66983,
 ('prep', 'adj'): 79621,
 ('adj', 'subst'): 188028,
 ('subst', 'prep'): 171005,
 ('prep', 'subst'): 323591,
 ('fin', 'subst'): 47463,
 ('ppas', 'prep'): 59418,
 ('adj', 'prep'): 56734,
 ('conj', 'brev'): 12454,
 ('fin', 'qub'): 41730,
 ('subst', 'adj'): 273260,
 ('qub', 'subst'): 17534,
 ('subst', 'subst'): 280316,
 ('fin', 'inf'): 16198,
 ('conj', 'prep'): 22306,
 ('conj', 'conj'): 5232,
 ('qub', 'brev'): 7729,
 ('qub', 'adv'): 6140,
 ('adv', 'prep'): 10405,
 ('qub', 'fin'): 13692,
 ('fin', 'adj'): 11839,
 ('subst', 'brev'): 12678,
 ('ger', 'prep'): 18753,
 ('qub', 'prep'): 14255,
 ('prep', 'adv'): 3182,
 ('subst', 'ger'): 42168,
 ('pact', 'prep'): 11696,
 ('qub', 'adj'): 5537,
 ('conj', 'adj'): 22833,
 ('fin', 'prep'): 15166,
 ('ppas', 'adv'): 3862,
 ('conj', 'ppron3'): 3802,
 ('qub', 'comp'): 1396,
 ('prep', 'ppron3'): 7993,
 ('conj', 'subst'): 83052,
 ('subst', 'ppas'): 45621,
 ('subst', 'fin'): 46233,
 ('adj', 'adj'): 14267,
 ('adj', 'qub'): 4690,
 ('ger'

# 9. Select the 10 largest partitions (partitions with the largest number of entries):

In [72]:
sorted_partitions = {k: v for k, v in sorted(llr_dict.items(), key=lambda item: item[1], reverse=True)[:10]}
sorted_partitions

{('prep', 'subst'): 323591,
 ('subst', 'subst'): 280316,
 ('subst', 'adj'): 273260,
 ('adj', 'subst'): 188028,
 ('subst', 'prep'): 171005,
 ('subst', 'conj'): 84094,
 ('conj', 'subst'): 83052,
 ('ger', 'subst'): 81237,
 ('prep', 'adj'): 79621,
 ('prep', 'brev'): 66983}

# 10. Use the computed LLR measure to select 5 bigrams for each of the largest categories:

In [74]:
sorted_llr = {k: v for k, v in sorted(llr_dict.items(), key=lambda item: item[1], reverse=True)}


In [85]:
part_llr_dict = dict()
for part in sorted_partitions.keys():
    print("\n", part)
    p1, p2 = part
    i = 0
    for b in sorted_llr.keys():
        if i < 5:
            c1, c2 = b.split(" ")[0].split(":")[1], b.split(" ")[1].split(":")[1]
            if p1 == c1 and p2 == c2:
                print(b)
                i += 1
                


 ('prep', 'subst')
z:prep dzień:subst
na:prep podstawa:subst
do:prep sprawa:subst
w:prep droga:subst
od:prep dzień:subst

 ('subst', 'subst')
droga:subst rozporządzenie:subst
skarb:subst państwo:subst
rada:subst minister:subst
terytorium:subst rzeczpospolita:subst
ochrona:subst środowisko:subst

 ('subst', 'adj')
minister:subst właściwy:adj
rzeczpospolita:subst polski:adj
jednostka:subst organizacyjny:adj
samorząd:subst terytorialny:adj
produkt:subst leczniczy:adj

 ('adj', 'subst')
który:adj mowa:subst
niniejszy:adj ustawa:subst
następujący:adj zmiana:subst
odrębny:adj przepis:subst
walny:adj zgromadzenie:subst

 ('subst', 'prep')
mowa:subst w:prep
ustawa:subst z:prep
wniosek:subst o:prep
dzień:subst od:prep
miesiąc:subst od:prep

 ('subst', 'conj')
przecinek:subst i:conj
wolność:subst albo:conj
imię:subst i:conj
całość:subst lub:conj
zasada:subst i:conj

 ('conj', 'subst')
i:conj tryb:subst
i:conj artykuł:subst
i:conj nazwisko:subst
i:conj usługa:subst
i:conj adres:subst

 ('ger', '

# 11. Using the results from the previous step answer the following questions:

* What types of bigrams have been found?

Najwięcej jest bigramów kategorii rzeownik + przyimek. Również często pojawiają sie bigramy typu rzeczownik + rzeczownik oraz rzeczownik + przymiotnik, co dobre pasuje do profilu tekstów ustaw.

* Which of the category-pairs indicate valuable multiword expressions? Do they have anything in common?

Najwięcej wystąpień mają pary kategorii: rzeczownik + przyimek, rzeczownik + rzeczownik i rzeczownik + przymiotnik. Jak można zauważyć w każdej parze pojawia się rzeczownik.

* Which signal: LLR score or syntactic category is more useful for determining genuine multiword expressions?

Jeśli miałbym wybrać jedno, to raczej LLR score (wyróżnienie wyrażeń dla danego profilu tekstu), jednak dobre wyniki da połączenie tych dwóch metod. Wówczas otrzymamy dość dobre narzędzie do wykrywania wyrażeń charakterystycznych dla danej dziedziny.

* Can you describe a different use-case where the morphosyntactic category is useful for resolving a real-world problem?

Tłumaczenia tekstów (lepsze dopasowanie słów w zależności od gramatyki) lub wykrywanie słów należących do danej dziedziny.